In [ ]:
import pandas as pd # for reading the data
from sklearn.preprocessing import MinMaxScaler # for data standardization
from sklearn.model_selection import train_test_split # to split the data into train and test parts
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder # to encode the data to numerical
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, \
f1_score, classification_report, cohen_kappa_score, matthews_corrcoef, log_loss, \
ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV # for cross validation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# A function that would plot for us the decision regions of a problem
# You have to install this package beforehand
from mlxtend.plotting import plot_decision_regions

# This library will be used to measure the difference in times between the start and the end of a process
import time
import numpy as np

## Data Preprocessing

#### Load the data in pandas data frame

In [ ]:
data_raw = pd.read_csv("data/social.csv", dtype = str)
data_raw.head()

#### Explore the data 

In [ ]:
#Explore the distribution of the two classes. 
#The two classes are slightly unbalanced, with the not-purchased class being around 0.65 of all targets. 
#Such inbalance is to be expected, in reality most of the people looking at an online add will not buy the product.
#In fact churn rates of 1 to 5% are most common. 
# Here 0 means no purchase
#      1 means purchased
data_raw['Purchased'].value_counts(normalize = True)

In [ ]:
#Explore the data types
#all the columns are of type object. This will not be ideal for the plotting at the end of the assignments. 
#That's why we need to cast them to numerical values.
print(data_raw.dtypes)

In [ ]:
#Before the data type transformation, we decide to drop the User ID and Gender column from our data,
# as they will likely not contribute to our decision
data = data_raw.drop(['User ID', 'Gender'], axis = 1)

In [ ]:
data

In [ ]:
#Cast remaining data columns to floats or int
data["Age"] = data["Age"].astype(float)
data["EstimatedSalary"] = data["EstimatedSalary"].astype(float)
data["Purchased"] = data["Purchased"].astype(int)

In [ ]:
#check to see if we've worked correctly
print(data.dtypes)

In [ ]:
#Check for missing values. There are none. This means we can proceed.
data.isnull().sum()

### Split the data into targets and inputs

In [ ]:
target = data['Purchased']
inputs = data.drop(['Purchased'],axis=1)

### Split the data into train and test sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs, target, test_size=0.3, random_state=365, stratify = target)

In [ ]:
#check the result of the stratification
pd.Series(y_train).value_counts(normalize = True)

In [ ]:
#Rescale the features to [-1, 1] for the support vector classification
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_train)
x_train_rescaled = scaling.transform(x_train)

### Classification with a linear SVM

In [ ]:
C = 1.0
svc = svm.SVC(kernel='linear', C=C)
svc.fit(x_train_rescaled, y_train) #train the svm

In [ ]:
y_pred_test = svc.predict(scaling.transform(x_test)) #predict on test data

### Analyze the results of the SVM

In [ ]:
#Analyze the results of the classifier first with the help of a confusion matrix
#The purchased class is larger by default,
#therefore, it is not surprising that there are more correctly assigned "no purchase" or class 0 instances.
#However, with a rate of almost two and half more correct values in the TP of class 0, it is clear,
#that this class is being recognised better from the classifer.

# Drop the seaborn style, so that we remove the white lines passing throguh the numbers 
sns.reset_orig()

fig, ax = plt.subplots(figsize=(8, 5))

cmp = ConfusionMatrixDisplay(
    confusion_matrix(y_test, y_pred_test),
    display_labels=["No Purchase", "Purchase"],
)

cmp.plot(ax=ax);

In [ ]:
#The classification report of the linear classifer.
#From the report we can see that the recall of the "purchase" class or class 0 is the lowest score, only 0.7.
#On the other hand the recall for the 0 class is 0.92, which is a great result
print(classification_report(y_test, y_pred_test, target_names = ["Not Purchased", "Purchased"]))

## Ploting decision boundaries

In [ ]:
knn_palette = sns.color_palette(['#FF5050', '#29757A'])
knn_palette

In [ ]:
%matplotlib inline
sns.set()

# Set the size of the figure
plt.figure(figsize = (16, 9))

# Create a scatter plot.
# On the x-axis, we have the first feature.
# The y-axis we have placed the second feature.
# The 'data' parameter specifies the dataset we are drawing the columns from.
# The 'hue' parameter specifies the feature based on which the points are going to be colored.
# The 'palette' parameter specifies the colors to be used in the plot.
# The 'markers' parameter determines the shape of the points.
# The 'style' parameter connects markers to classes.
# The 's' parameter specifies the size of the points.
# The 'alpha' parameter controls the opacity of the datapoints.
# We have decided to set the 'legend' parameter equal to False as we will make no use of it in this example.
sns.scatterplot(x = 'Age', y = 'EstimatedSalary', 
                data = data, 
                hue = 'Purchased', 
                palette = knn_palette,
                markers = ['^', 'P'],
                style = 'Purchased', 
                s = 100, 
                alpha = 1,
                legend = False);

In [ ]:
# plot the decision regions

start = time.time()

plt.figure(figsize = (16, 9))

plot_decision_regions(X = x_train_rescaled, y = y_train.to_numpy(), #cast the y_train variable to a numpy array
                      clf = svc,
                      markers = ['^', 'P'],
                      colors = '#ff5050,#29757a',
                      scatter_kwargs = {'s':100, 'edgecolor':'white', 'alpha':1},
                      legend = 2)
;

end = time.time()

### Polynomial kernel

In [ ]:
svc_poly = svm.SVC(kernel='poly', C=10)
svc_poly.fit(x_train_rescaled, y_train)

In [ ]:
# plot the decision regions

start = time.time()

plt.figure(figsize = (16, 9))

plot_decision_regions(X = x_train_rescaled, y = y_train.to_numpy(),
                      clf = svc_poly,
                      markers = ['^', 'P'],
                      colors = '#ff5050,#29757a',
                      scatter_kwargs = {'s':100, 'edgecolor':'white', 'alpha':1},
                      legend = 2)
;

end = time.time()

### Sigmoid kernel

In [ ]:
svc_sigmoid = svm.SVC(kernel='sigmoid', C=10)
svc_sigmoid.fit(x_train_rescaled, y_train)

In [ ]:
# plot the decision regions

start = time.time()

plt.figure(figsize = (16, 9))

plot_decision_regions(X = x_train_rescaled, y = y_train.to_numpy(),
                      clf = svc_sigmoid,
                      markers = ['^', 'P'],
                      colors = '#ff5050,#29757a',
                      scatter_kwargs = {'s':100, 'edgecolor':'white', 'alpha':1},
                      legend = 2)
;

end = time.time()

### Radial kernel

In [ ]:
svc_radial = svm.SVC(kernel='rbf', C=10)
svc_radial.fit(x_train_rescaled, y_train)

In [ ]:
# plot the decision regions

start = time.time()

plt.figure(figsize = (16, 9))

plot_decision_regions(X = x_train_rescaled, y = y_train.to_numpy(),
                      clf = svc_radial,
                      markers = ['^', 'P'],
                      colors = '#ff5050,#29757a',
                      scatter_kwargs = {'s':100, 'edgecolor':'white', 'alpha':1},
                      legend = 2)
;

end = time.time()